# Antara

In [56]:
import requests
from bs4 import BeautifulSoup
import csv
import random

def filter_ads(soup, ad_classes):
    ad_divs = soup.find_all(class_= ad_classes) 
    for ad_div in ad_divs:
        ad_div.extract()

def filter_text(text):
    unwanted_texts = ['Jakarta (ANTARA/JACX) – ', 'Jakarta (ANTARA) -']
    
    filtered_text = text
    for unwanted_text in unwanted_texts:
        filtered_text = filtered_text.replace(unwanted_text, '')

    return filtered_text

def extract_titles_and_urls(soup):
    return soup.find_all(class_='post_title post_title_medium')

def extract_title(entry):
    title_text = entry.text.strip()
    return title_text

def extract_url(entry):
    url_tag = entry.find('a')
    if url_tag:
        return url_tag.get('href')
    else:
        return ""
    
def extract_text(url):
    news_page = requests.get(url)
    news_soup = BeautifulSoup(news_page.content, 'html.parser')

    # remove bold
    bold_texts = news_soup.find_all('b')
    for bold_text in bold_texts:
        bold_text.extract()

    text_content = news_soup.find('div', class_='wrap__article-detail-content post-content')
    filter_ads(news_soup , "text-muted mt-2 small")

    if text_content:
        text1 = text_content.text.strip()
        text = filter_text(text1)
        return text
    else:
        return "News content not found for: " + url

train_csv_file = open('training_data2.csv', 'w', newline='', encoding='utf-8')
test_csv_file = open('Fake.csv', 'w', newline='', encoding='utf-8')

train_csv_writer = csv.writer(train_csv_file)
test_csv_writer = csv.writer(test_csv_file)

train_csv_writer.writerow(['Title', 'URL', 'Text', 'Label'])
test_csv_writer.writerow(['Title', 'URL', 'Text'])

num_pages = 30
for page in range(1, num_pages + 1):
    url = f"https://www.antaranews.com/slug/anti-hoax/{page}"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    entries = extract_titles_and_urls(soup)
    random.shuffle(entries)

    # Split the data into training (80%) and testing (20%) datasets
    num_train = int(0.8 * len(entries))
    train_entries = entries[:num_train]
    test_entries = entries[num_train:]

    # Keywords
    keywords = ["anies", "prabowo", "presiden", "cawapres", "pemilu", "ganjar", "caslon", "gibran", "tps", "mahfud", "muhaimin", "cak imin", "kpu", "pilpres", "amin"]  # Add more keywords as needed

    # training
    for entry in train_entries:
        title = extract_title(entry)
        url = extract_url(entry)
        keyword_found = False
        for keyword in keywords:
            if keyword in title.lower() and keyword in url.lower():
                keyword_found = True
                break
        if keyword_found:
            text = extract_text(url)
            train_csv_writer.writerow([title, url, text, 0]) 
        else:
            continue

    # test
    for entry in test_entries:
        title = extract_title(entry)
        url = extract_url(entry)
        keyword_found = False
        for keyword in keywords:
            if keyword in title.lower() and keyword in url.lower():
                keyword_found = True
                break
        if keyword_found:
            text = extract_text(url)
            test_csv_writer.writerow([title, url, text])
        else:
            continue

train_csv_file.close()
test_csv_file.close()


In [23]:
import requests
from bs4 import BeautifulSoup
import csv
import random

def filter_ads(soup, ad_classes):
    ad_divs = soup.find_all(class_= ad_classes) 
    for ad_div in ad_divs:
        ad_div.extract()

def filter_text(text):
    unwanted_texts = ['Jakarta (ANTARA/JACX) – ', 'Jakarta (ANTARA) -']
    
    filtered_text = text
    for unwanted_text in unwanted_texts:
        filtered_text = filtered_text.replace(unwanted_text, '')

    return filtered_text

def extract_titles_and_urls(soup):
    return soup.find_all(class_='post_title post_title_medium')

def extract_title(entry):
    title_text = entry.text.strip()
    return title_text

def extract_url(entry):
    url_tag = entry.find('a')
    if url_tag:
        return url_tag.get('href')
    else:
        return ""
    
def extract_text(url):
    news_page = requests.get(url)
    news_soup = BeautifulSoup(news_page.content, 'html.parser')

    # remove bold
    bold_texts = news_soup.find_all('b')
    for bold_text in bold_texts:
        bold_text.extract()

    text_content = news_soup.find('div', class_='wrap__article-detail-content post-content')
    filter_ads(news_soup , "text-muted mt-2 small")

    if text_content:
        text1 = text_content.text.strip()
        text = filter_text(text1)
        return text
    else:
        return "News content not found for: " + url

def extract_complete_date(date_element):
    span_element = date_element.text.strip()
    # print(span_element)
    if span_element:
        if "Kemarin" in span_element:
            print(span_element)
        if "jam lalu" in span_element:
            print(span_element)

num_pages = 1
for page in range(1, num_pages + 1):
    url = f"https://www.antaranews.com/slug/anti-hoax/{page}"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    entries = extract_titles_and_urls(soup)
    filter_ads(soup, 'col-md-4')
    date_elements = soup.find_all(class_='text-secondary')

    for entry, date_element in zip(entries, date_elements):
        extract_complete_date(date_element)



Kemarin 21:34
Kemarin 14:57
Kemarin 12:39
Kemarin 09:23


clean dataset

In [7]:
import csv

def filter_text(text):
    filtered_text = text
    filtered_text = ' '.join(filtered_text.split())
    return filtered_text.strip()


unwanted_strings = ['Hoaks! ', 'Disinformasi! ', 'Misinformasi! ']

def remove_salah_from_title(text):
    for unwanted_string in unwanted_strings:
        text = text.replace(unwanted_string, '')
    return text.strip() 

def filter_csv(input_file, output_file):
    with open(input_file, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
            writer = csv.writer(outfile)
            for row in reader:
                row[0] = remove_salah_from_title(row[0])
                row[2] = filter_text(row[2])
                writer.writerow(row)

input_file = 'training_data2.csv'
output_file = 'Cleaned/training_data2.csv'

input_file2 ="Fake.csv"
output_file2 = 'Cleaned/Fake.csv'

filter_csv(input_file, output_file)
filter_csv(input_file2, output_file2)